In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date, timedelta
from dotenv import load_dotenv
import os
import time

# carregando as variáveis de ambiente
load_dotenv()


True

In [91]:
def retornar_periodo():
    hoje = date.today()
    amanha = hoje + timedelta(days=1)
    hoje_str = date.strftime(hoje, '%d/%m/%Y')
    amanha_str = date.strftime(amanha, '%d/%m/%Y')
    periodo_str = (f'{hoje_str} - {amanha_str}')
    return periodo_str


In [90]:
def obter_driver():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # maximixando a tela
    driver.maximize_window()
    return driver

In [89]:
def logar(driver):
    # indo para a tela de login
    driver.get("https://qualylab.gerencialab.com.br/")

    # aguardando elemento aparecer na tela
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div/div[1]/a/img'))
        )
    time.sleep(4)

    # obtendo as variáveis de ambiente
    login = os.getenv('LOGIN')
    password = os.getenv('PASSWORD')

    # preenchendo o campo usuário
    driver.find_element(By.XPATH, '//*[@id="loginsite"]').send_keys(login)

    # preenchendo o campo senha
    driver.find_element(By.XPATH, '//*[@id="senhasite"]').send_keys(password)

    # clicando em acessar
    driver.find_element(By.XPATH, '//*[@id="authLogin"]').click()

    # aguardando logo GerenciaLab Aparecer na tela
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[1]/div/a/img')))
    time.sleep(4)

    return True

True


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x008EFC03+61635]
	GetHandleVerifier [0x008EFC44+61700]
	(No symbol) [0x007105D3]
	(No symbol) [0x0075899E]
	(No symbol) [0x00758D3B]
	(No symbol) [0x007A0E12]
	(No symbol) [0x0077D2E4]
	(No symbol) [0x0079E61B]
	(No symbol) [0x0077D096]
	(No symbol) [0x0074C840]
	(No symbol) [0x0074D6A4]
	GetHandleVerifier [0x00B74523+2701795]
	GetHandleVerifier [0x00B6FCA6+2683238]
	GetHandleVerifier [0x00B8A9EE+2793134]
	GetHandleVerifier [0x009068C5+155013]
	GetHandleVerifier [0x0090CFAD+181357]
	GetHandleVerifier [0x008F7458+92440]
	GetHandleVerifier [0x008F7600+92864]
	GetHandleVerifier [0x008E1FF0+5296]
	BaseThreadInitThunk [0x77235D49+25]
	RtlInitializeExceptionChain [0x77E7D09B+107]
	RtlGetAppContainerNamedObjectPath [0x77E7D021+561]


In [26]:
# indo para ordens de serviço
driver.get("https://qualylab.gerencialab.com.br/service-order")

In [45]:
# aguardando label Ordem serviços aparecer na tela
elemento = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-wrapper"]/div/div/h4')))
time.sleep(4)

In [49]:
# clicando no botão Limpar Pesquisa
driver.find_element(By.XPATH, '//*[@id="ButtonsShowTable"]/div/button[5]').click()

In [ ]:
# obtendo o campo da data de entrega
campo_data = driver.find_element(By.XPATH, '//*[@id="dataPrazoEntregaOSForm"]')

In [50]:
campo_data.send_keys(retornar_periodo())

In [51]:
from selenium.webdriver.common.keys import Keys

# clicando em aplicar
driver.find_element(By.XPATH, '/html/body/div[5]/div[4]/button[2]').click()

In [72]:
lista_amostras = driver.find_elements(By.CSS_SELECTOR, '#tableOrdemdeServico tr.even, #tableOrdemdeServico tr.odd')

In [73]:
amostras = []
for linha in lista_amostras:
    cliente = linha.find_elements(By.TAG_NAME, 'td')[4].text
    status_os = linha.find_elements(By.TAG_NAME, 'td')[1].text
    amostra = linha.find_elements(By.TAG_NAME, 'td')[2].text
    prioridade = linha.find_elements(By.TAG_NAME, 'td')[3].text
    amostras.append((status_os, amostra, cliente, prioridade))